<a href="https://colab.research.google.com/github/pin164/Automated-Expense-Categorization/blob/main/Random_forest_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# STEP 1
# Import  dependencies
!pip install category_encoders
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn import tree
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import Callback
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
import category_encoders as ce

from google.colab import drive
drive.mount('/content/drive')



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 3.0 MB/s eta 0:00:00
Mounted at /content/drive


## Preprocessing

In [ ]:
# STEP 2
#  Import and read the Cleaned_Invoice_4.csv
#/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_4.csv
application_df = pd.read_csv("/content/drive/MyDrive/Project 4/Automated-Expense-Categorization/Cleaned_Invoice_4.csv",encoding='latin1').iloc[:,:14]
application_df.head()

,Vendor,Month,Year,PO#,Invoice Date,Invoice#,total Invoice Amt,TRS,Project,Amount,Cost Centre,Order#,Dept
0,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089109,WSH LINK Replacement - D&I,$455.00,165239,300007147,FIN
1,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090144,AIMS Replacement (PDO-217),"$3,575.00",171318,930089792,BTT
2,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7089063,Petrinex - D&I,$877.50,190230,300007150,ARD
3,vendor 1039,Apr,2021,4501190248,2021-04-30,4,"$18,850.00",SDS7090248,M3P Vitality Upgrade - Implementation Phase - ...,"$4,062.50",195006,300007300,JUS
4,vendor 1039,Apr,2021,4501180577,2021-04-30,5,"$18,850.00",SDS7089423,Civil Legal Billing System - Scoping,"$5,005.00",171318,300007265,JUS


In [ ]:
# STEP 3
# extract beneficial columns only
####X_df = application_df[['Column2','PO#','TRS','Order#','Column3']]
X_df = application_df[['PO#','TRS','Order#']]
Y_df = application_df['Cost Centre']



In [ ]:
#Step 4
# amend the data type of order to be an object
X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')

X_df.dtypes

<ipython-input-4-78b2616e01c1>:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[300007147 930089792 300007150 ... 300007069 300007069 300007069]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_df.loc[:, 'Order#'] = X_df['Order#'].astype('object')


,0
PO#,object
TRS,object
Order#,object


In [ ]:
#Step 5
# Ordinal encoding transforms the columns PO#','TRS','Order#'
# into a column of class integers.
# Ordinal encoding scheme maps columns to integers randomly


encoder = ce.OrdinalEncoder()
encoder.fit_transform(X_df).head()

,PO#,TRS,Order#
0,1,1,1
1,1,2,2
2,1,3,3
3,1,4,4
4,2,5,5


In [ ]:
X_encoded = encoder.fit_transform(X_df) # Store the encoded DataFrame in X_encoded

In [ ]:
#Drop Rows with Missing Target Values:
#remove the rows from both the feature set X and the target set Y_df that contain missing target values.
# Combine X and y to drop rows with missing target values
combined_df = pd.concat([X_encoded, Y_df], axis=1)

# Drop rows where the target variable is missing
combined_df.dropna(subset=['Cost Centre'], inplace=True)

# Separate the features and target again after dropping missing values
X = combined_df.drop(columns=['Cost Centre']).values
y = combined_df['Cost Centre'].values


# Re-split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the resulting datasets to verify the split
print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)
print("Training Target Shape:", y_train.shape)
print("Testing Target Shape:", y_test.shape)



Training Features Shape: (5822, 3)
Testing Features Shape: (1456, 3)
Training Target Shape: (5822,)
Testing Target Shape: (1456,)


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Initialize the model
rf_model = RandomForestClassifier(random_state=42)

In [ ]:
#Some Cost Centres dominate the dataset,
# the model could struggle to predict minority payees.


# Using Class Weights:
rf_model = RandomForestClassifier(class_weight='balanced', random_state=42)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
print(classification_report(y_test, y_pred, zero_division=1))



              precision    recall  f1-score   support

      110508       1.00      1.00      1.00         8
      110889       1.00      1.00      1.00        13
      111173       1.00      1.00      1.00        15
      111508       1.00      1.00      1.00        21
      135072       0.92      1.00      0.96        12
      135139       1.00      1.00      1.00         5
      135150       1.00      1.00      1.00        13
      135367       1.00      1.00      1.00         7
      140010       1.00      1.00      1.00        15
      140548       1.00      1.00      1.00        19
      140642       1.00      1.00      1.00         4
      160207       1.00      1.00      1.00         2
      160781       1.00      1.00      1.00         3
      160996       1.00      1.00      1.00         4
      161017       1.00      1.00      1.00        10
      163124       1.00      1.00      1.00         5
      165031       0.90      1.00      0.95         9
      165039       1.00    

In [ ]:
rf_confusion = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(rf_confusion)

Confusion Matrix:
[[ 8  0  0 ...  0  0  0]
 [ 0 13  0 ...  0  0  0]
 [ 0  0 15 ...  0  0  0]
 ...
 [ 0  0  0 ...  7  0  0]
 [ 0  0  0 ...  0 16  0]
 [ 0  0  0 ...  0  0  9]]


In [ ]:
# Assuming clf is your trained RandomForestClassifier
import joblib
joblib.dump(rf_model, 'rf_model.pkl')

['rf_model.pkl']

In [28]:
# Save encoder as well the way we saved the model
joblib.dump(encoder, 'category_encoder.pkl')



['category_encoder.pkl']

streamlit code

In [ ]:
create a def

In [29]:
import pandas as pd
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
import joblib
# Load the model from the file
clf_loaded = joblib.load('/content/rf_model.pkl')
# Load the encoder from the file
encoder_loaded = joblib.load('/content/category_encoder.pkl')

In [20]:
clf_loaded

RandomForestClassifier(class_weight='balanced', random_state=42)

In [30]:
encoder_loaded

OrdinalEncoder(cols=['PO#', 'TRS', 'Order#'],
               mapping=[{'col': 'PO#', 'data_type': dtype('O'),
                         'mapping': 4501190248         1
4501180577         2
4501174431         3
4501200163         4
4501215854         5
                ... 
NoPO-Janthony    434
4501130820       435
4501138122       436
4501151341       437
NaN               -2
Length: 438, dtype: int64},
                        {'col': 'TRS', 'data_type': dtype('O'),
                         'mapping': SDS7089109      1
SDS7090144      2
SDS7089063      3
SDS7090248      4
SDS7089423      5
             ... 
CIO7090261    225
SDS7090328    226
SDS7089436    227
SDS7089062    228
NaN            -2
Length: 229, dtype: int64},
                        {'col': 'Order#', 'data_type': dtype('O'),
                         'mapping': 300007147.0      1
930089792.0      2
300007150.0      3
300007300.0      4
300007265.0      5
              ... 
220003600.0    169
930092582.0    170
930083878.0    171
200040308.0    172
NaN             -2
Length: 173, dtype: int64}])

In [31]:
test =pd.DataFrame(columns= ['PO#','TRS','Order#'])

step one in streamlit

In [21]:
po = input("Enter PO#: ")
trs = input("Enter TRS: ")
order = input("Enter Order#: ")


Enter PO#: 4501190248
Enter TRS: SDS7090432
Enter Order#: 930093243


In [33]:
test.loc[0] = [po,trs,order]

In [34]:
test

,PO#,TRS,Order#
0,4501190248,SDS7090432,930093243


In [35]:
# Assuming you have new data (X_test) to transform
X_test_transformed = encoder_loaded.transform(test)

In [36]:
X_test_transformed

,PO#,TRS,Order#
0,1,161,-1.0


In [37]:
clf_loaded.predict(X_test_transformed)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:486: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


array([171318])